# Code for Stirling reps of the first and second kinds

**WARNING: THE FUNCTIONS FOR THE ORLIK-SOLOMON ALGEBRA MAY NOT WORK IF YOU ARE NOT USING SAGEMATH 10.0 OR LATER**

Run the first cell with NO CHANGES.

Later cells contain demos for obtaining families of representations for OS and VG of the Type A braid arrangement.

Functions include:

1. `koszulDualReps(primalReps, q)` : input a list of symmetric group characters corresponding to graded pieces of an algebra $A$. Outputs a list of characters for the Koszul dual up to degree $q$.
2. `stirlingFirstVG(n,k)`
3. `stirlingFirstOS(n,k)`
4. `stirlingSecondVG(n,k)`
5. `stirlingSecondOS(n,k)`
6. `rankTwoRepsOS(n)` : Characters for the graded pieces of $OS(U_{2,n})$
7. `rankTwoDualRepsOS(n,q)`: Characters for the graded pieces of $OS(U_{2,n})^!$ up to degree $q$
8. `equivariantDeviationsSkew(L)` : input a list of characters for the graded pieces of A^! up to degree n WHERE A IS **SKEW-COMMUTATIVE**. Outputs list of chracters for primitives / indecomposables.
9. `equivariantDeviationsCommutative(L)` input a list of characters for the graded pieces of A^! up to degree n WHERE A IS **COMMUTATIVE**. Outputs list of characters for primitives / indecomposable.

In [15]:
########################
### DECLARATIONS ###
########################

Sym = SymmetricFunctions(QQ);

# need to pick a basis to do anything meaningful
s = Sym.s() # Schur basis
h = Sym.h() # homogeneous basis
m = Sym.m() # monomial basis
p = Sym.p() # power sum basis
e = Sym.e() # elementary basis


# declare basis w.r.t. homogeneous polynomials for defining H
#R.<t,z> = PowerSeriesRing(SymmetricFunctions(QQ).h())

############################################################################


##########################################
### KOSZUL DUAL REPRESENTATIONS ###
##########################################

#INPUT: A list of representations L, and a number p up to which we will compute the duals
#OUTPUT: The representations for the Koszul dual [A_i]^!
#CAVEAT: The output only makes sense and is only guaranteed to be schur-positive if A is Koszul

def koszulDualReps(primalReps,q):
    returnedReps = [];
    n = len(primalReps);
    for i in range(q): 
        if i == 0: 
            returnedReps.append(s([n]))
            continue 
        if i == 1: 
            returnedReps.append(primalReps[1])
            continue
        totalRep = 0
        for j in range(1,min(i,n)): 
            sgn = (-1)**(j+1)
            totalRep += sgn * primalReps[j].itensor(returnedReps[i-j])
        if (i < n):
            totalRep += (-1)**(i+1) * primalReps[i]
        returnedReps.append(totalRep)
    return returnedReps


##########################################
### STIRLING REPRESENTATIONS ###
##########################################

#INPUT: integers n,k
#OUTPUT: The Stirling rep c_{VG}(n,k) of the first kind
def stirlingFirstVG(n,k):
    L = Partitions(n,length=n-k).list();
    return sum(s.gessel_reutenauer(x) for x in L);

#INPUT: integers n,k
#OUTPUT: The Stirling rep c_{OS}(n,k) of the first kind
def stirlingFirstOS(n,k):   
    L = Partitions(n,length=n-k).list();
    return sum(s.lehrer_solomon(x) for x in L)

#INPUT: integers n,p
#OUTPUT: A list of Stirling reps S_{OS}(n,i) of the second kind from 0 to p
#q = how far out to compute to
def stirlingSecondOS(n,q):
    returnedReps = [];
    for i in range(q): 
        if i == 0: 
            returnedReps.append(s([n]))
            continue 
        primalReps = [stirlingFirstOS(n,k) for k in range(0,n)];
        return koszulDualReps(primalReps,q)

#INPUT: integers n,p
#OUTPUT: A list of Stirling reps S_{VG}(n,i) of the second kind from 0 to p
#q = how far out to compute to
def stirlingSecondVG(n,q):
    returnedReps = [];
    for i in range(q): 
        if i == 0: 
            returnedReps.append(s([n]))
            continue 
        primalReps = [stirlingFirstVG(n,k) for k in range(0,n)];
        return koszulDualReps(primalReps,q)

############################
### RANK TWO MATROIDS ###
############################

#INPUT: n the size of the ground set
#OUTPUT: Representations for the graded pieces of OS(U_{2,n})
def rankTwoRepsOS(n):
    return [s[n],s[n]+s[n-1,1],s[n-1,1]]

#INPUT: n the size of the ground set
#OUTPUT: Representations for the graded pieces of OS(U_{2,n})
def rankTwoDualRepsOS(n,q):
    return koszulDualReps(rankTwoRepsOS(n),q)


#################################
### EQUIVARIANT DEVIATIONS ###
#################################

#INPUT: A list L of representations for the graded pieces of A^! up to degree n WHERE A IS **SKEW-COMMUTATIVE**
#OUTPUT: The equivariant deviations up to degree n
def equivariantDeviationsSkew(L):
    n = len(L);
    returnedReps = [0,L[1]];
    for i in range(2,n): 
        totalRep = L[i];
        for q in Partitions(i, min_length=2):
            expPart = q.to_exp()
            tensorList = [];
            for j in range(0,len(expPart)):
                tensorList.append(s[expPart[j]].inner_plethysm(returnedReps[j+1]));
            repTensor = tensorList[0];
            if (len(tensorList)>1):
                for j in range (1,len(tensorList)):
                    repTensor = repTensor.itensor(tensorList[j]);
            totalRep += -repTensor
        returnedReps.append(totalRep)
    return returnedReps

#INPUT: A list L of representations for the graded pieces of A^! up to degree n WHERE A IS **COMMUTATIVE**
#OUTPUT: The equivariant deviations up to degree n
def equivariantDeviationsCommutative(L):
    n = len(L);
    returnedReps = [0,L[1]];
    for i in range(2,n): 
        totalRep = L[i];
        for q in Partitions(i, min_length=2):
            expPart = q.to_exp()
            tensorList = [];
            for j in range(0,len(expPart)):
                if (j%2==1):
                    tensorList.append(s[expPart[j]].inner_plethysm(returnedReps[j+1]));
                if (j%2==0):
                    tensorList.append(e[expPart[j]].inner_plethysm(returnedReps[j+1]));
            repTensor = tensorList[0];
            if (len(tensorList)>1):
                for j in range (1,len(tensorList)):
                    repTensor = repTensor.itensor(tensorList[j]);
            totalRep += -repTensor
        returnedReps.append(totalRep)
    return returnedReps

In [16]:
#loop for displaying rank 2 matroid equivariant deviations
a = 6
n = 6
eqDevs = [];
for i in range(2,n): 
    eqDevs.append(equivariantDeviationsSkew(rankTwoDualRepsOS(i,a)));

for j in range(2,a):
    print("equivariant deviations for j = " + str(j) + ":")
    for k in range(len(eqDevs)):
              print(" n = " + str(k+2) + ": " + str((eqDevs[k][j])))

equivariant deviations for j = 2:
 n = 2: 0
 n = 3: s[1, 1, 1]
 n = 4: s[2, 1, 1]
 n = 5: s[3, 1, 1]
equivariant deviations for j = 3:
 n = 2: 0
 n = 3: s[2, 1]
 n = 4: s[2, 1, 1] + s[2, 2] + s[3, 1]
 n = 5: s[2, 2, 1] + s[3, 1, 1] + s[3, 2] + s[4, 1]
equivariant deviations for j = 4:
 n = 2: 0
 n = 3: s[1, 1, 1] + s[2, 1]
 n = 4: s[1, 1, 1, 1] + 3*s[2, 1, 1] + s[2, 2] + 2*s[3, 1]
 n = 5: 2*s[2, 1, 1, 1] + 2*s[2, 2, 1] + 4*s[3, 1, 1] + 2*s[3, 2] + 2*s[4, 1]
equivariant deviations for j = 5:
 n = 2: 0
 n = 3: s[1, 1, 1] + 2*s[2, 1] + s[3]
 n = 4: 2*s[1, 1, 1, 1] + 6*s[2, 1, 1] + 4*s[2, 2] + 6*s[3, 1] + 2*s[4]
 n = 5: s[1, 1, 1, 1, 1] + 6*s[2, 1, 1, 1] + 8*s[2, 2, 1] + 10*s[3, 1, 1] + 9*s[3, 2] + 8*s[4, 1] + 2*s[5]


In [18]:
#loop for displaying VG equivariant deviations
n = 8
a = 7
eqDevs = [];
for i in range(2,n): 
    eqDevs.append(equivariantDeviationsCommutative(stirlingSecondVG(i,a)));

for j in range(1,a):
    print("equivariant deviations for j = " + str(j) + ":")
    for k in range(len(eqDevs)):
              print(" n = " + str(k+2) + ": " + str((eqDevs[k][j])))

equivariant deviations for j = 1:
 n = 2: s[1, 1]
 n = 3: s[1, 1, 1] + s[2, 1]
 n = 4: s[2, 1, 1] + s[3, 1]
 n = 5: s[3, 1, 1] + s[4, 1]
 n = 6: s[4, 1, 1] + s[5, 1]
 n = 7: s[5, 1, 1] + s[6, 1]
equivariant deviations for j = 2:
 n = 2: s[2]
 n = 3: s[2, 1] + 2*s[3]
 n = 4: s[2, 2] + 2*s[3, 1] + 2*s[4]
 n = 5: 2*s[3, 2] + 2*s[4, 1] + 2*s[5]
 n = 6: s[3, 3] + 2*s[4, 2] + 2*s[5, 1] + 2*s[6]
 n = 7: s[4, 3] + 2*s[5, 2] + 2*s[6, 1] + 2*s[7]
equivariant deviations for j = 3:
 n = 2: 0
 n = 3: s[2, 1]
 n = 4: s[2, 1, 1] + 2*s[2, 2] + s[3, 1]
 n = 5: 2*s[2, 2, 1] + s[3, 1, 1] + 2*s[3, 2] + s[4, 1]
 n = 6: s[2, 2, 2] + 2*s[3, 2, 1] + s[4, 1, 1] + 2*s[4, 2] + s[5, 1]
 n = 7: s[3, 2, 2] + 2*s[4, 2, 1] + s[5, 1, 1] + 2*s[5, 2] + s[6, 1]
equivariant deviations for j = 4:
 n = 2: 0
 n = 3: s[1, 1, 1] + s[2, 1]
 n = 4: s[1, 1, 1, 1] + 4*s[2, 1, 1] + s[2, 2] + 2*s[3, 1]
 n = 5: 3*s[2, 1, 1, 1] + 3*s[2, 2, 1] + 6*s[3, 1, 1] + 2*s[3, 2] + 2*s[4, 1]
 n = 6: 2*s[2, 2, 1, 1] + 4*s[3, 1, 1, 1] + 5*s[3, 2, 

In [19]:
#loop for displaying OS equivariant deviations
#NOTE: This function may not work for you if you are not using SageMath 10.0 or higher
n = 6
a = 6
eqDevs = [];
for i in range(2,n): 
    eqDevs.append(equivariantDeviationsSkew(stirlingSecondOS(i,a)));

for j in range(2,a):
    print("equivariant deviations for j = " + str(j) + ":")
    for k in range(len(eqDevs)):
              print(" n = " + str(k+2) + ": " + str(eqDevs[k][j]))

AttributeError: 'SymmetricFunctionAlgebra_schur_with_category' object has no attribute 'lehrer_solomon'

In [21]:
#loop for displaying VG reps
n = 7
l = 6
for i in range(1, n+1): 
    print("representations of VG(A_" + str(i-1) + "):")
    reps = [];
    for j in range(0,l) :
      reps.append(stirlingFirstVG(i,j));
    for k in range(len(reps)): 
        print("    degree " + str(k) + ": " + str(reps[k]))

representations of VG(A_0):
    degree 0: s[1]
    degree 1: 0
    degree 2: 0
    degree 3: 0
    degree 4: 0
    degree 5: 0
representations of VG(A_1):
    degree 0: s[2]
    degree 1: s[1, 1]
    degree 2: 0
    degree 3: 0
    degree 4: 0
    degree 5: 0
representations of VG(A_2):
    degree 0: s[3]
    degree 1: s[1, 1, 1] + s[2, 1]
    degree 2: s[2, 1]
    degree 3: 0
    degree 4: 0
    degree 5: 0
representations of VG(A_3):
    degree 0: s[4]
    degree 1: s[2, 1, 1] + s[3, 1]
    degree 2: s[1, 1, 1, 1] + s[2, 1, 1] + 2*s[2, 2] + s[3, 1]
    degree 3: s[2, 1, 1] + s[3, 1]
    degree 4: 0
    degree 5: 0
representations of VG(A_4):
    degree 0: s[5]
    degree 1: s[3, 1, 1] + s[4, 1]
    degree 2: s[1, 1, 1, 1, 1] + s[2, 1, 1, 1] + 2*s[2, 2, 1] + s[3, 1, 1] + 2*s[3, 2] + s[4, 1]
    degree 3: 2*s[2, 1, 1, 1] + 2*s[2, 2, 1] + 3*s[3, 1, 1] + 2*s[3, 2] + s[4, 1]
    degree 4: s[2, 1, 1, 1] + s[2, 2, 1] + s[3, 1, 1] + s[3, 2] + s[4, 1]
    degree 5: 0
representations of VG(A_5

In [23]:
#Lists OS(A_n)^! for first n
#NOTE: This may not work for you if you are not using SageMath 10.0 or higher
n = 7
l = 8
for j in range(2, n+1): 
    print("representations of OS(A_" + str(j-1) + ")^!:")
    reps = stirlingSecondOS(j,l);
    for k in range(len(reps)): 
        print("    degree " + str(k+1) + ": " + str(reps[k]))

representations of OS(A_1)^!:


AttributeError: 'SymmetricFunctionAlgebra_schur_with_category' object has no attribute 'lehrer_solomon'

In [22]:
#Lists VG(A_n)^! for first n
n = 9
for j in range(2, n+1): 
    print("representations of VG(A_" + str(j-1) + ")^!:")
    reps = stirlingSecondVG(j,n);
    for k in range(len(reps)): 
        print("    degree " + str(k) + ": " + str(reps[k]))

representations of VG(A_1)^!:
    degree 0: s[2]
    degree 1: s[1, 1]
    degree 2: s[2]
    degree 3: s[1, 1]
    degree 4: s[2]
    degree 5: s[1, 1]
    degree 6: s[2]
    degree 7: s[1, 1]
    degree 8: s[2]
representations of VG(A_2)^!:
    degree 0: s[3]
    degree 1: s[1, 1, 1] + s[2, 1]
    degree 2: s[1, 1, 1] + 2*s[2, 1] + 2*s[3]
    degree 3: 3*s[1, 1, 1] + 5*s[2, 1] + 2*s[3]
    degree 4: 5*s[1, 1, 1] + 10*s[2, 1] + 6*s[3]
    degree 5: 11*s[1, 1, 1] + 21*s[2, 1] + 10*s[3]
    degree 6: 21*s[1, 1, 1] + 42*s[2, 1] + 22*s[3]
    degree 7: 43*s[1, 1, 1] + 85*s[2, 1] + 42*s[3]
    degree 8: 85*s[1, 1, 1] + 170*s[2, 1] + 86*s[3]
representations of VG(A_3)^!:
    degree 0: s[4]
    degree 1: s[2, 1, 1] + s[3, 1]
    degree 2: s[1, 1, 1, 1] + 3*s[2, 1, 1] + 2*s[2, 2] + 3*s[3, 1] + 2*s[4]
    degree 3: 4*s[1, 1, 1, 1] + 11*s[2, 1, 1] + 8*s[2, 2] + 11*s[3, 1] + 4*s[4]
    degree 4: 12*s[1, 1, 1, 1] + 38*s[2, 1, 1] + 24*s[2, 2] + 38*s[3, 1] + 13*s[4]
    degree 5: 40*s[1, 1, 1, 1] +

    degree 0: s[9]
    degree 1: s[7, 1, 1] + s[8, 1]
    degree 2: s[5, 2, 1, 1] + 2*s[6, 1, 1, 1] + 2*s[6, 2, 1] + s[6, 3] + 3*s[7, 1, 1] + 3*s[7, 2] + 3*s[8, 1] + 2*s[9]
    degree 3: s[3, 2, 2, 2] + s[3, 3, 1, 1, 1] + s[4, 1, 1, 1, 1, 1] + 3*s[4, 2, 1, 1, 1] + 3*s[4, 2, 2, 1] + 3*s[4, 3, 1, 1] + s[4, 3, 2] + s[4, 4, 1] + 6*s[5, 1, 1, 1, 1] + 11*s[5, 2, 1, 1] + 8*s[5, 2, 2] + 8*s[5, 3, 1] + 3*s[5, 4] + 13*s[6, 1, 1, 1] + 22*s[6, 2, 1] + 10*s[6, 3] + 18*s[7, 1, 1] + 18*s[7, 2] + 14*s[8, 1] + 4*s[9]
    degree 4: s[2, 2, 1, 1, 1, 1, 1] + 2*s[2, 2, 2, 1, 1, 1] + 2*s[2, 2, 2, 2, 1] + 3*s[3, 1, 1, 1, 1, 1, 1] + 12*s[3, 2, 1, 1, 1, 1] + 20*s[3, 2, 2, 1, 1] + 10*s[3, 2, 2, 2] + 16*s[3, 3, 1, 1, 1] + 22*s[3, 3, 2, 1] + 5*s[3, 3, 3] + 18*s[4, 1, 1, 1, 1, 1] + 57*s[4, 2, 1, 1, 1] + 62*s[4, 2, 2, 1] + 66*s[4, 3, 1, 1] + 49*s[4, 3, 2] + 31*s[4, 4, 1] + 52*s[5, 1, 1, 1, 1] + 140*s[5, 2, 1, 1] + 84*s[5, 2, 2] + 122*s[5, 3, 1] + 35*s[5, 4] + 100*s[6, 1, 1, 1] + 185*s[6, 2, 1] + 88*s[6, 3] + 113*s[